- Get latent space for amphiphiles using VICGAE

In [1]:
import sys
sys.path.append("../src/models")
sys.path.append("../src")

In [2]:
from astrochem_embedding import VICGAE
import torch

In [14]:
import pandas as pd
import numpy as np
from VICGAE_latent import *

In [12]:
model_vicgae = VICGAE.from_pretrained()

In [8]:
df_structured_ori = pd.read_csv('../../dataweight_function_data/df_structured_ori.csv')
df_structured_mf = pd.read_csv('../../dataweight_function_data/df_structured_mol_frac.csv')

In [9]:
def binarize_last_column(df):
    """
    Converts the last column of a DataFrame to binary values:
    1 if value > 0, else 0.
    Returns a modified copy of the DataFrame.
    """
    df_bin = df.copy()
    last_col = df_bin.columns[-1]
    df_bin[last_col] = (df_bin[last_col] > 0).astype(int)
    return df_bin

In [10]:
def prepare_latent_dataset_VICGAE(df_structured, model):
    """
    Prepare latent dataset with VICGAE FM model

    """
    df_structured_classify = binarize_last_column(df_structured)
    
    x_latent_am, y_am = build_mixture_latent_features_VICGAE(
        df=df_structured_classify,
        smi_cols=['smi1', 'smi2', 'smi3', 'smi4', 'smi5', 'smi6', 'smi7'],
        conc_cols=['conc1', 'conc2', 'conc3', 'conc4', 'conc5', 'conc6', 'conc7'],
        target_col='vesicles_formation',
        latent_fn=latent_fn_VICGAE,
        latent_fn_args={"model": model}
    )

    df_total = pd.concat([pd.DataFrame(x_latent_am), pd.DataFrame(y_am, columns=['vesicles_formation'])],axis=1)
 
    return df_total

In [16]:
df_total_ori = prepare_latent_dataset_VICGAE(df_structured_ori, model_vicgae)
df_total_mf = prepare_latent_dataset_VICGAE(df_structured_mf, model_vicgae)

In [19]:
df_total_ori.to_csv('../../dataweight_function_data/df_total_amphiphiles_VICGAE_wf_ori.csv', index=False)
df_total_mf.to_csv('../../dataweight_function_data/df_total_amphiphiles_VICGAE_wf_mf.csv', index=False)